In [3]:
#conda install -c ambermd pytraj
#conda install -c bioconda nglview
#jupyter-nbextension enable nglview --py --sys-prefix
#%matplotlib widget

In [ ]:
#interactive 
import pytraj as pt
import nglview as nv
from subprocess import call
with open("polyA.pdb", "w") as f:
    call(["./fd_helix", "abdna","aaaaaaaaaa"], stdout=f)
traj = pt.load('polyA.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

In [2]:
#conda install -c omnia openmm
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

pdb = PDBFile('polyA.pdb')
pdb.getTopology()

In [5]:
modeller = Modeller (pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens (forcefield)
modeller.getTopology()

<Topology; 2 chains, 20 residues, 638 atoms, 686 bonds>

In [6]:
pdbfile.PDBFile.writeModel(modeller.topology,modeller.positions,open('polyA_with_H.pdb','w'))
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view.add_ball_and_stick('not protein')
view

NGLWidget()

In [6]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)
system = forcefield.createSystem(modeller.topology)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
#simulation.minimizeEnergy()

In [7]:
simulation.reporters.append(PDBReporter('polyA_with_H.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True,potentialEnergy=True, temperature=True))
simulation.step(3000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,7040.9327794529245,243.01056162362443
1000,6778.741831626034,303.2860580009655
1500,6531.520544483888,318.20409088669817
2000,6378.362160636245,320.81211537117406
2500,6016.904381118435,315.1465192263966
3000,5848.989507700089,319.0844973795195


In [8]:
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view.add_ball_and_stick('not protein')
view

NGLWidget(count=30)

In [9]:
import parmed as pmd
pdbfile = PDBFile('polyA_with_H.pdb')
structure = pmd.openmm.topsystem.load_topology( pdbfile.topology, system, pdbfile.positions)
structure.save('polyA_with_H.prmtop', overwrite=True)
structure.save('polyA_with_H.inpcrd', format='rst7', overwrite=True)
prmtop = AmberPrmtopFile('polyA_with_H.prmtop')
inpcrd = AmberInpcrdFile('polyA_with_H.inpcrd')

In [10]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)
system = prmtop.createSystem(implicitSolvent=HCT)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

/home/mahdi_moqri/anaconda3/envs/nglview/lib/python3.6/site-packages/simtk/openmm/app/internal/amber_file_parser.py:1019: UserWarning: Non-optimal GB parameters detected for GB model HCT
  'Non-optimal GB parameters detected for GB model %s' % gbmodel)


In [11]:
simulation.reporters.append(PDBReporter('polyA_with_H.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True,potentialEnergy=True, temperature=True))
simulation.step(3000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,959865.4957894722,243.04609158814398
1000,959819.0947681121,283.3169190045422
1500,959927.8028189342,295.25839653091015
2000,960064.2913005712,282.5264487474633
2500,960089.6395222802,291.7759616795708
3000,959950.5212129389,302.26002950795856


In [12]:
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view.add_ball_and_stick('not protein')
view

NGLWidget(count=30)

In [13]:
modeller.addSolvent(forcefield, boxSize=Vec3(5.0, 3.5, 3.5)*nanometers)

In [14]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.001*picoseconds)
system = forcefield.createSystem(modeller.topology)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
#simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('polyA_with_H.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True,potentialEnergy=True, temperature=True))
simulation.step(3000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,-82661.56195945281,364.7470196162332
1000,-85785.6421217677,366.0999355207831
1500,-87298.96207504386,346.92900578000007
2000,-88605.44512947416,335.52958116706264
2500,-90285.79737571432,334.5668581647169
3000,-91181.86733946975,318.077263492028


In [15]:
traj = pt.load('polyA_with_H.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view.add_ball_and_stick('not protein')
view

NGLWidget(count=30)